In [2]:
import boto3
from dotenv import load_dotenv
import os
import re
import paramiko
import requests

In [2]:
load_dotenv()
aws_access_key_id= os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")

# Let's use Amazon S3
ec2 = boto3.client('ec2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

In [12]:
kwargs['InstanceType']

't3.micro'

In [7]:
response = ec2.describe_instances(InstanceIds=[instance_id])
instance_status = response['Reservations'][0]['Instances'][0]['State']['Name']
print(f"Instance {instance_id} is {instance_status}.")

Instance i-0a82804a4d4ee930b is running.


In [8]:
response = ec2.terminate_instances(InstanceIds=[instance_id])

In [10]:
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print(f"Instance {instance_id} termination request successful.")
else:
    print(f"Failed to terminate instance {instance_id}.")

Instance i-0a82804a4d4ee930b termination request successful.


In [11]:
while True:
    response = ec2.describe_instances(InstanceIds=[instance_id])
    instance_status = response['Reservations'][0]['Instances'][0]['State']['Name']
    if instance_status == 'terminated':
        print(f"Instance {instance_id} terminated successfully.")
        break
    else:
        print(f"Instance {instance_id} is still {instance_status}. Waiting...")
        time.sleep(5)

Instance i-0a82804a4d4ee930b terminated successfully.


In [9]:
response

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0a82804a4d4ee930b',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '2110a1a0-047d-4483-a8c3-8b9f4dee6ea0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2110a1a0-047d-4483-a8c3-8b9f4dee6ea0',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Fri, 02 Jun 2023 14:57:36 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [17]:
load_dotenv()
# login to aws
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")
BUCKET_NAME = os.getenv("aws_s3_bucket")

COPERNICUS_API_USER = os.getenv("COPERNICUS_API_USER")
COPERNICUS_API_SECRET = os.getenv("COPERNICUS_API_SECRET")
COPERNICUS_API_URL = os.getenv("COPERNICUS_API_URL")

DOCKERHUB_USER = os.getenv("DOCKERHUB_USER")
DOCKERHUB_PASSWORD = os.getenv("DOCKERHUB_PASSWORD")

PROJECT_NAME = os.getenv("PROJECT_NAME")

In [19]:
EC2_KWARGS = {
    "ImageId": "ami-07151644aeb34558a",  # ID des Amazon Linux 2 AMI
    "InstanceType": "t3.micro",  # Instanztyp t3.medium
    "MinCount": 1,  # Mindestanzahl von Instanzen
    "MaxCount": 1,  # Maximale Anzahl von Instanzen
    "KeyName": "Ec2-boto3",  # Name des Schlüsselpaars
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],  # ID der Sicherheitsgruppe
    "BlockDeviceMappings": [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "VolumeSize": 20,  # Größe des Root-Volumes in GB
                "DeleteOnTermination": True,
                "VolumeType": "gp3",
            },
        },
    ],
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                echo {DOCKERHUB_PASSWORD} | sudo docker login --username {DOCKERHUB_USER} --password-stdin
                sudo docker pull {DOCKERHUB_USER}/{PROJECT_NAME}-SERVICE_PLACEHOLDER:latest
                sudo docker run \
                -e DOCKERIZED=True \
                -e aws_access_key_id={aws_access_key_id} \
                -e aws_secret_access_key={aws_secret_access_key} \
                -e region_name={region_name} \
                -e aws_s3_bucket={BUCKET_NAME} \
                -e COPERNICUS_API_USER={COPERNICUS_API_USER} \
                -e COPERNICUS_API_SECRET={COPERNICUS_API_SECRET} \
                -e COPERNICUS_API_URL={COPERNICUS_API_URL} \
                {DOCKERHUB_USER}/{PROJECT_NAME}-SERVICE_PLACEHOLDER:latest
             """,  # Skript, das beim Start der Instanz ausgeführt wird
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": f"{PROJECT_NAME}-SERVICE_PLACEHOLDER"},
            ],
        },
    ],
}

In [32]:
EC2_KWARGS["UserData"] = EC2_KWARGS["UserData"].replace(
    'SERVICE_PLACEHOLDER', 'downloader'
)

In [27]:
EC2_KWARGS["TagSpecifications"][0]["Tags"][0]["Value"] = EC2_KWARGS["TagSpecifications"][0]["Tags"][0]["Value"].replace(

[{'ResourceType': 'instance',
  'Tags': [{'Key': 'Name',
    'Value': 'solar-park-detection-SERVICE_PLACEHOLDER'}]}]

In [24]:
EC2_KWARGS["TagSpecifications"] = EC2_KWARGS["TagSpecifications"].replace(
    "SERVICE_PLACEHOLDER", service
)

str

In [29]:
EC2_KWARGS["TagSpecifications"][0]["Tags"][0]["Value"] = EC2_KWARGS["TagSpecifications"][0]["Tags"][0]["Value"].replace(

'solar-park-detection-SERVICE_PLACEHOLDER'

In [32]:
import requests
response = requests.get('http://localhost:8081/models')
response.json()['models'][0]

{'modelName': 'solar-park-detection', 'modelUrl': 'solar-park-detection.mar'}

In [30]:
response = requests.get('http://localhost:8081/models')
response.headers

{'content-type': 'application/json', 'x-request-id': '670d9fac-daeb-432f-bfa2-983aa78fe0a3', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET, POST, PUT, OPTIONS', 'access-control-allow-headers': 'Content-Type', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache; no-store, must-revalidate, private', 'Expires': 'Thu, 01 Jan 1970 00:00:00 UTC', 'content-length': '122', 'connection': 'keep-alive'}

In [17]:
EC2_KWARGS = {
    "ImageId": "ami-07151644aeb34558a",  # ID des Amazon Linux 2 AMI
    "InstanceType": "t2.micro",  # Instanztyp t3.medium
    "MinCount": 1,  # Mindestanzahl von Instanzen
    "MaxCount": 1,  # Maximale Anzahl von Instanzen
    "KeyName": "Ec2-boto3",  # Name des Schlüsselpaars
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],  # ID der Sicherheitsgruppe
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "BlockDeviceMappings": [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "VolumeSize": 20,  # Größe des Root-Volumes in GB
                "DeleteOnTermination": True,
                "VolumeType": "gp3",
            },
        },
    ],
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo curl \
                    -L https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m) \
                    -o /usr/local/bin/docker-compose
                sleep 3    
                sudo chmod +x /usr/local/bin/docker-compose
                sudo docker pull taraman12/solar-park-detection-api:latest
                sudo docker pull postgres:latest
             """,  # Skript, das beim Start der Instanz ausgeführt wird
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": f"api"},
            ],
        },
    ],
}

In [18]:
response = ec2.run_instances(**EC2_KWARGS)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-0d5547ce8b7670c8c


In [5]:
response['Instances'][0]

{'AmiLaunchIndex': 0,
 'ImageId': 'ami-07151644aeb34558a',
 'InstanceId': 'i-0e8a9d8b30d446637',
 'InstanceType': 't2.micro',
 'KeyName': 'Ec2-boto3',
 'LaunchTime': datetime.datetime(2023, 6, 6, 19, 59, 42, tzinfo=tzutc()),
 'Monitoring': {'State': 'disabled'},
 'Placement': {'AvailabilityZone': 'eu-central-1a',
  'GroupName': '',
  'Tenancy': 'default'},
 'PrivateDnsName': 'ip-172-31-29-249.eu-central-1.compute.internal',
 'PrivateIpAddress': '172.31.29.249',
 'ProductCodes': [],
 'PublicDnsName': '',
 'State': {'Code': 0, 'Name': 'pending'},
 'StateTransitionReason': '',
 'SubnetId': 'subnet-0e19e3f6d404a4467',
 'VpcId': 'vpc-07332de596f81bf40',
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [],
 'ClientToken': '9bf006d4-0763-4829-9a33-30be072d6ed0',
 'EbsOptimized': False,
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2023, 6, 6, 19, 59, 42, tzinfo=tzutc()),
    'AttachmentId': 'eni-attach-03bcd69110afbc814',
 

In [10]:
private_ip = response['Instances'][0]['PrivateIpAddress']

In [30]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('3.122.112.217', username='ec2-user', key_filename='Ec2-boto3.pem')

In [31]:
# Kopieren Sie die Docker Compose-Datei auf die Instanz
sftp = ssh.open_sftp()
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\.env', '/home/ec2-user/.env')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\docker-compose.yml', '/home/ec2-user/docker-compose.yml')
sftp.close()

In [24]:
ssh

In [23]:
stdin, stdout, stderr = ssh.exec_command('sudo docker-compose up')

In [26]:
stdin, stdout, stderr

(<paramiko.ChannelFile from <paramiko.Channel 4 (closed) -> <paramiko.Transport at 0xcb5d5750 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 4 (closed) -> <paramiko.Transport at 0xcb5d5750 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 4 (closed) -> <paramiko.Transport at 0xcb5d5750 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>)

In [40]:
output = stdout.read().decode('utf-8')
print(output)

In [41]:
output

''

In [42]:
stdin, stdout, stderr = ssh.exec_command('cd /path/to/docker-compose.yml && sudo docker-compose up -d')

In [5]:
# Starten Sie die EC2-Instanz
response = ec2.run_instances(**kwargs)

NameError: name 'kwargs' is not defined

In [13]:
EC2_KWARGS = {
    "ImageId": "ami-07151644aeb34558a",  # ID des Amazon Linux 2 AMI
    "InstanceType": "t3.medium",  # Instanztyp t3.medium
    "MinCount": 1,  # Mindestanzahl von Instanzen
    "MaxCount": 1,  # Maximale Anzahl von Instanzen
    "KeyName": "Ec2-boto3",  # Name des Schlüsselpaars
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },  # ID der Sicherheitsgruppe
    "BlockDeviceMappings": [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "VolumeSize": 20,  # Größe des Root-Volumes in GB
                "DeleteOnTermination": True,
                "VolumeType": "gp3",
            },
        },
    ],
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo docker pull taraman12/solar-park-detection-preprocessing:latest
                sudo docker pull taraman12/solar-park-detection-ml-serve:latest
                sudo docker run \
                --cpus 1 \
                -p 8080:8080 \
                -p 8081:8081 \
                --expose 8080 \
                --expose 8081 \
                --name ml-serve \
                --network host \
                taraman12/solar-park-detection-ml-serve:latest
             """,  # Skript, das beim Start der Instanz ausgeführt wird
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": f"solar-park-detection-ml-serve"},
            ],
        },
    ],
}

In [14]:
response = ec2.run_instances(**EC2_KWARGS)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-095ed9046e70d097f


In [ ]:
                    --link ml-serve
               -e DOCKERIZED=True \
                -e MAKE_TRAININGS_DATA=False \
                -e PRODUCTION=True \

sudo docker run --rm -e DOCKERIZED=True -p 8000:8000 --expose 8000 --name api taraman12/solarpark-detection-api:latest

Allow all traffic (check http instead of https)

docker swarm init --advertise-addr <ip address of the manager node>


sudo docker service create --name db -e POSTGRES_PASSWORD="CarcW*Go42h79tV6T" -e POSTGRES_USER="taraman" -e POSTGRES_DB="solar-park-detection" -e POSTGRES_HOST="db" -e POSTGRES_PORT="5432" postgres

sudo docker service create --name api -e DOCKERIZED=true -e POSTGRES_PASSWORD="CarcW*Go42h79tV6T" -e POSTGRES_USER="taraman" -e POSTGRES_DB="solar-park-detection" -e POSTGRES_HOST="db" -e POSTGRES_PORT="5432" -p 8000:8000 taraman12/solar-park-detection-api:latest

sudo docker service create --cpus 1 -p 8080:8080 -p 8081:8081 --name ml-serve taraman12/solar-park-detection-ml-serve:latest

Test Lambda connection

In [19]:
import requests
import json
import torch
import numpy as np


# URL des Endpunkts Ihrer AWS Lambda-Funktion
url = "https://5y4jvfx1vc.execute-api.eu-central-1.amazonaws.com/default/Test-ml-serve"

In [6]:
data_array = torch.randn(1, 4, 256, 256)
HEADERS = {"Content-Type": "application/json"}

In [20]:
    json_data = json.dumps(data_array.tolist())
    req = requests.post(
        f"{url}/predictions/solar-park-detection",
        headers=HEADERS,
        data=json_data,
    )
    print(f"Got response from ml model: {req.status_code}")
    pred = np.array(req.json())

Got response from ml model: 403


In [21]:
pred

array({'message': 'Missing Authentication Token'}, dtype=object)